In [1]:
%pylab qt

Populating the interactive namespace from numpy and matplotlib


In [133]:
from region.image import Image
from utils import *
import numpy
import skimage.filters
from skimage.draw import polygon_perimeter, polygon
from scipy.interpolate import interp2d, dfitpack

In [150]:
np.set_printoptions(suppress=True)
# numpy.set_printoptions(threshold=numpy.nan)
numpy.set_printoptions(threshold=1000)

# Utils

In [56]:
def getPoints(im, n):
    n = int(n)
    plt.imshow(im.imArr)
    points = plt.ginput(n, timeout = 300, show_clicks=True)
    plt.close()
    print("Points gotten from", im.name, ":", str(points))
    return np.array(points)

def dispPtsOnImg(im, results, tag=False):
    dispPtsOnImgRC(im, results[:, 0], results[:, 1], tag)

def dispPtsOnImgRC(im, rowInds, colInds, tag=False):
    assert len(rowInds) == len(colInds)
    plt.imshow(im)
    plt.plot(rowInds, colInds, 'bo')
    if tag:
        for ind in range(len(rowInds)):
            x, y = rowInds[ind], colInds[ind]
            plt.text(x, y, str(ind), bbox=dict(facecolor='blue', alpha=0.5))
            plt.text(x, y, "%d-%d" % (x,y), bbox=dict(facecolor='red', alpha=0.5))
    plt.waitforbuttonpress()
    plt.close()

In [119]:
def computeH(startPts, endPts):
    assert len(startPts) == len(endPts)
    A, b = createAb(startPts, endPts)
    
    x = numpy.linalg.lstsq(A, b)
    result = np.append(x[0], 1)
    return result.reshape(3, 3)

In [120]:
def createAxxx(startPts, endPts):
    result = np.zeros((len(startPts) * 2 + 1, 9))
    for i in range(len(startPts)):
        startPoint = startPts[i]
        endPoint = endPts[i]
        
        # first row set
        result[i*2, 0] = -1 * startPoint[0]
        result[i*2, 1] = -1 * startPoint[1]
        result[i*2, 2] = -1
        
        result[i*2, 6] = startPoint[0] * endPoint[0]
        result[i*2, 7] = startPoint[1] * endPoint[0]
        result[i*2, 8] = endPoint[0]
        
        
        # second row set
        result[i*2+1, 3] = -1 * startPoint[0]
        result[i*2+1, 4] = -1 * startPoint[1]
        result[i*2+1, 5] = -1
        
        result[i*2+1, 6] = startPoint[0] * endPoint[1]
        result[i*2+1, 7] = startPoint[1] * endPoint[1]
        result[i*2+1, 8] = endPoint[1]
    result[-1, -1] = 1
    
    return result


def createAb(startPts, endPts):
    result = np.zeros((len(startPts) * 2, 8))
    b = np.zeros(len(result))
    for i in range(len(startPts)):
        startPoint = startPts[i]
        endPoint = endPts[i]
        
        # first row set
        result[i*2, 0] = -1 * startPoint[0]
        result[i*2, 1] = -1 * startPoint[1]
        result[i*2, 2] = -1
        
        result[i*2, 6] = startPoint[0] * endPoint[0]
        result[i*2, 7] = startPoint[1] * endPoint[0]
        b[i*2] = -endPoint[0]
        
        
        # second row set
        result[i*2+1, 3] = -1 * startPoint[0]
        result[i*2+1, 4] = -1 * startPoint[1]
        result[i*2+1, 5] = -1
        
        result[i*2+1, 6] = startPoint[0] * endPoint[1]
        result[i*2+1, 7] = startPoint[1] * endPoint[1]
        b[i*2+1] = -endPoint[1]
    # result[-1, -1] = 1
    
    return result, b
    

# def createA(numOfPts):
#     result = np.zeros((numOfPts * 2, 8))
    
#     for i in range()

# def createA(numOfPts):
#     result = np.zeros((numOfPts * 2, 8))
    
#     for i in range()

In [ ]:
dispPtsOnImg(imSproul0.imArr, imSproul0.pts)
dispPtsOnImg(imSproul1.imArr, imSproul1.pts)

In [ ]:
"""
p' = H p
where
    p' = target
    p = source

place on p' the color on p

---

get the points in final image that actually will need a value from source image
    do by calculating the results of the corners of the source img getting transformed,
    then using polygon to get the coordinates that actually lies inside the polygon, so you dont do the coords that isnt in it
    

for every point in p', get the corresponding p with p' * H^-1

"""

In [ ]:
sourcePts = imSproul0.pts
targetPts = imSproul1.pts

H = computeH(sourcePts, targetPts)
print("source:", sourcePts)
print("transformed source:", transf(H, sourcePts))
print("target:", targetPts)

dispPtsOnImg(imSproul0.imArr, sourcePts)
dispPtsOnImg(imSproul1.imArr, transf(H, sourcePts))

In [9]:
def transf(H, res, normalize=True):
    if res.ndim == 1:
        vect = np.append(res, 1)
    else:
        vect = np.vstack((res.T, np.ones(len(res))))
    tfSourceCorners = np.dot(H, vect)
    
    if normalize:
        for i in range(tfSourceCorners.shape[1]):
            corner = tfSourceCorners[:, i]
            normCorner = np.dot(corner, 1/(corner[2]))
            tfSourceCorners[:, i] = normCorner
    return tfSourceCorners[:-1, :].T

In [10]:
def getDimensions(imCornersCoords):
    # assuming imCornersCoords = [[width, height]]
    
    minWidth = int(np.floor(min(imCornersCoords[:, 0])))
    minHeight = int(np.floor(min(imCornersCoords[:, 1])))
    
    maxWidth = int(np.ceil(max(imCornersCoords[:, 0])))
    maxHeight = int(np.ceil(max(imCornersCoords[:, 1])))
    
    totalWidth = maxWidth - minWidth + 1
    totalHeight = maxHeight - minHeight + 1
    
    tup = totalHeight, totalWidth, minHeight, minWidth
    print(tup)
    
    return tup
    

In [112]:
def findAllCoords(tfSourceCorners):
    
    xCorners = tfSourceCorners[:, 0]
    yCorners = tfSourceCorners[:, 1]
    
    flatMinX = int(np.floor(min(xCorners)))
    flatMinY = int(np.floor(min(yCorners)))
    newXCorners = xCorners - flatMinX
    newYCorners = yCorners - flatMinY
    
    
    innerCoordsR, innerCoordsC = polygon(newXCorners, newYCorners)
    finInnerCoordsR, finInnerCoordsC = innerCoordsR + flatMinX, innerCoordsC + flatMinY
    
    
    perimCoordsR, perimCoordsC = polygon_perimeter(xCorners, yCorners)
    allCoordsR = np.append(finInnerCoordsR, perimCoordsR)
    allCoordsC = np.append(finInnerCoordsC, perimCoordsC)

#     allCoordsR = finInnerCoordsR
#     allCoordsC = finInnerCoordsC

#     allCoordsR = perimCoordsR
#     allCoordsC = perimCoordsC

    targetPs = np.vstack((allCoordsR, allCoordsC)).T
    
    return targetPs

In [107]:
class WarpedImageInfo:
    
    def __init__(self, corners, targetPs, rvals, gvals, bvals):
        self.corners = corners
        self.targetPs = targetPs
        self.rvals = rvals
        self.gvals = gvals
        self.bvals = bvals
        
    def createImageObj(self, newImage=None, minH = 0, minW = 0, name = ""):
        if newImage == None:
            finH, finW, minH, minW = getDimensions(self.corners)
            newImage = Image(name, np.zeros((finH, finW, 3)))
        newImage.imArr[self.targetPs[:, 1] - minH, self.targetPs[:, 0] - minW, 0] = self.rvals
        newImage.imArr[self.targetPs[:, 1] - minH, self.targetPs[:, 0] - minW, 1] = self.gvals
        newImage.imArr[self.targetPs[:, 1] - minH, self.targetPs[:, 0] - minW, 2] = self.bvals
        return newImage

def warpTo(imSource, sourcePts, targetPts):

    H = computeH(sourcePts, targetPts)
    sourceCorners = Image.getCorners(imSource)
    tfSourceCorners = transf(H, sourceCorners)

    print("sourceCorners:", sourceCorners)
    print("tfSourceCorners:", tfSourceCorners)

    targetPs = findAllCoords(tfSourceCorners)

    invH = np.linalg.inv(H)

    sourcePs = transf(invH, targetPs)
    
    source_Rchan = interp2d(range(imSource.width), range(imSource.height), imSource.imArr[:, :, 0])
    source_Gchan = interp2d(range(imSource.width), range(imSource.height), imSource.imArr[:, :, 1])
    source_Bchan = interp2d(range(imSource.width), range(imSource.height), imSource.imArr[:, :, 2])
    autoInterp = lambda f, xs, ys: dfitpack.bispeu(f.tck[0], f.tck[1], f.tck[2], f.tck[3], f.tck[4], xs, ys)[0]

    Rvals = autoInterp(source_Rchan, sourcePs[:, 0], sourcePs[:, 1])
    Gvals = autoInterp(source_Gchan, sourcePs[:, 0], sourcePs[:, 1])
    Bvals = autoInterp(source_Bchan, sourcePs[:, 0], sourcePs[:, 1])
    
    
    return WarpedImageInfo(tfSourceCorners, targetPs, Rvals, Gvals, Bvals)

def warpProcess(imBase, imBasePts, imgList):

    warpedResults = []
    for im in imgList:

        imPts = im.pts

        warpedResults.append(warpTo(im, imPts, imBasePts))

    allCorners = np.vstack([warped.corners for warped in warpedResults])
    print(allCorners)
    finH, finW, minH, minW = getDimensions(allCorners)
    # finalImage = Image("finalImage", np.zeros((finH, finW, 3)))

    newWarpedImages = []
    # for warped in warpedResults:
    for i in range(len(warpedResults)):
        warped = warpedResults[i]
        newImage = Image("finalImage", np.zeros((finH, finW, 3)))
        newWarpedImages.append(warped.createImageObj(newImage, minH = minH, minW = minW))
    
    return newWarpedImages

# Laplacian blending

In [184]:
def multiResBlendOp(im1, im2, mask, levels, sigma):
    assert im1.shape == im2.shape == mask.shape

    L1 = laplacianPyrOp_3D(im1, levels, sigma)
    L2 = laplacianPyrOp_3D(im2, levels, sigma)
    LM = gaussStackOp_3D(mask, levels, sigma)
    
#     for i in range(len(LM)):
#         printImage(str(i) + ".png", LM[i])
        
    for i in range(len(L1)):
        printImage(str(i) + ".png", L1[i])
    LM1 = LM
    LM2 = (1 - LM1)

    L1_post = LM1 * L1
    L2_post = LM2 * L2

    finalL = L1_post + L2_post

    tes = np.zeros(L1[0].shape)

    for i in range(len(L1)):
        tes += finalL[i]
        break

    tes2 = np.clip(tes, -1, 1)

    return tes2

In [123]:
def laplacianPyrOp_3D(im, levels, sigma, scaleB = False):
    gaussStack = gaussStackOp_3D(im, levels, sigma)

    for i in range(levels):
        res = gaussStack[i] - gaussStack[i+1]
        if scaleB:
            finalCurrLayer = (res - res.min()) / (res.max() - res.min())
        else:
            finalCurrLayer = res
        gaussStack[i] = finalCurrLayer

    return gaussStack

In [188]:
def gaussStackOp_3D(im, levels, sigma):
    assert levels > 0
    #inclusive of original img, at layer indexed 0

    result = []
    # newLayer = (lambda: np.zeros(im.shape))

    for i in range(levels+1):
        if i == 0:
            result.append(im)
            continue
        # currLayer = newLayer()
        currLayer = skimage.filters.gaussian(result[i - 1], sigma, mode="constant")
        result.append(currLayer)

    return np.array(result)

In [141]:
imMask = Image.fromPath("../curated_sample_imgs/lower_sproul/mask_test.png", 0.25)
res = multiResBlendOp(imSproul0.imArr, imSproul1.imArr, imMask.imArr, 3, 5)

/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/filters/_gaussian.py:108: RuntimeWarning: Images with dimensions (M, N, 3) are interpreted as 2D+RGB by default. Use `multichannel=False` to interpret as 3D image with last dimension of length 3.
  warn(RuntimeWarning(msg))


In [142]:
printImage("test.png", res)

File saved to: output_imgs/20181110184925_test.png


/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [189]:
im1 = result[0].imArr
im2 = result[1].imArr

imMask = np.zeros(im1.shape)
# im1 == 0

imMask[np.any(im1, 2), 0] = 1
imMask[np.any(im1, 2), 1] = 1
imMask[np.any(im1, 2), 2] = 1

res = multiResBlendOp(im1, im2, imMask, 3, 5)

printImage("test.png", res)

/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/filters/_gaussian.py:108: RuntimeWarning: Images with dimensions (M, N, 3) are interpreted as 2D+RGB by default. Use `multichannel=False` to interpret as 3D image with last dimension of length 3.
  warn(RuntimeWarning(msg))
/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: output_imgs/20181110194111_1.png is a low contrast image
  warn('%s is a low contrast image' % fname)


File saved to: output_imgs/20181110194111_0.png
File saved to: output_imgs/20181110194111_1.png


/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: output_imgs/20181110194111_2.png is a low contrast image
  warn('%s is a low contrast image' % fname)


File saved to: output_imgs/20181110194111_2.png
File saved to: output_imgs/20181110194111_3.png
File saved to: output_imgs/20181110194112_test.png


In [138]:
res = laplacianPyrOp_3D(imSproul0.imArr, 10, 1, True)

/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/filters/_gaussian.py:108: RuntimeWarning: Images with dimensions (M, N, 3) are interpreted as 2D+RGB by default. Use `multichannel=False` to interpret as 3D image with last dimension of length 3.
  warn(RuntimeWarning(msg))


In [139]:
for i in range(len(res)):
    img = res[i]
    printImage(str(i) + ".jpg", img)

/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


File saved to: output_imgs/20181110184344_0.jpg
File saved to: output_imgs/20181110184344_1.jpg
File saved to: output_imgs/20181110184344_2.jpg
File saved to: output_imgs/20181110184344_3.jpg
File saved to: output_imgs/20181110184344_4.jpg
File saved to: output_imgs/20181110184344_5.jpg
File saved to: output_imgs/20181110184344_6.jpg
File saved to: output_imgs/20181110184344_7.jpg
File saved to: output_imgs/20181110184344_8.jpg
File saved to: output_imgs/20181110184344_9.jpg
File saved to: output_imgs/20181110184344_10.jpg


# Scenarios

In [152]:
imSproul0 = Image.fromPath("../curated_sample_imgs/lower_sproul/lower_sproul_00.jpg", 0.25)
imSproul0.readCoordsIn("../curated_sample_imgs/lower_sproul/lower_sproul_00.txt")
imSproul0.pts = imSproul0.pts * 0.25
imSproul1 = Image.fromPath("../curated_sample_imgs/lower_sproul/lower_sproul_01.jpg", 0.25)
imSproul1.readCoordsIn("../curated_sample_imgs/lower_sproul/lower_sproul_01.txt")
imSproul1.pts = imSproul1.pts * 0.25

imgList = [imSproul0, imSproul1]
imBase = imSproul1
imBasePts = imSproul1.pts

result = warpProcess(imBase, imBasePts, imgList)
for i in range(len(result)):
    im = result[i]
    printImage(str(i) + ".jpg", im.imArr)

/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """


sourceCorners: [[   0    0]
 [   0  489]
 [1007  489]
 [1007    0]]
tfSourceCorners: [[ 419.60439299   95.02282132]
 [ 354.93081049  446.20618062]
 [1843.20307759  644.81057364]
 [1325.84944889   41.8996847 ]]
sourceCorners: [[   0    0]
 [   0  489]
 [1007  489]
 [1007    0]]
tfSourceCorners: [[  -0.   -0.]
 [   0.  489.]
 [1007.  489.]
 [1007.   -0.]]
[[ 419.60439299   95.02282132]
 [ 354.93081049  446.20618062]
 [1843.20307759  644.81057364]
 [1325.84944889   41.8996847 ]
 [  -0.           -0.        ]
 [   0.          489.        ]
 [1007.          489.        ]
 [1007.           -0.        ]]
(647, 1846, -1, -1)
-1 -1
File saved to: output_imgs/20181110190602_0.jpg
File saved to: output_imgs/20181110190603_1.jpg


/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [60]:
imRect = Image.fromPath("../curated_sample_imgs/lower_sproul/lower_sproul_00.jpg", 0.25)
imRect.readCoordsIn("../curated_sample_imgs/lower_sproul/rect_orig_lower_sproul_00.txt")
imRect.pts = imRect.pts * 0.25
dispPtsOnImg(imRect.imArr, imRect.pts)

targPts = Image.readCoords("../curated_sample_imgs/lower_sproul/rect_targ_lower_sproul_00.txt")
targPts = targPts * 0.25
print(targPts)
dispPtsOnImg(imRect.imArr, targPts)

res = warpTo(imRect, imRect.pts, targPts)
im = res.createImageObj()
printImage("rectified.jpg", im.imArr)

/Users/reginaongowarsito/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


[[  0.    0. ]
 [  0.  130. ]
 [130.  137.5]
 [137.5   0. ]]
